In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
dat=pd.read_excel("./AuthorList.xlsx",engine='openpyxl').set_index('id')


In [3]:
#FirstAuthorIDs=[51,79,105,13,21,32,71,74,101]
FirstAuthorIDs=[]

In [4]:
#Lift from excel sheet which columns are institutions and addresses.
# Anything labeled "Institution" is an institution column and the one to its
# right is an address column.
ar=["Institution" in i for i in dat.columns]
InstColumns=np.array(dat.columns)[ar]
AddrColumns=np.array(dat.columns)[np.concatenate([[False],ar[:-1]])]
AddrDict={InstColumns[i]:AddrColumns[i] for i in range(0,len(InstColumns))}

In [5]:
# Put the author list and list of institutions in the correct order
OrderOfAuthors=[]
UniqueInstitutions=[]
Addresses=[]
for i in FirstAuthorIDs:
    OrderOfAuthors.append(i)
    for InstCol in InstColumns:
        if(dat.loc[i].notna()[InstCol] and not dat.loc[i][InstCol] in UniqueInstitutions):
            UniqueInstitutions.append(dat.loc[i][InstCol])
            Addresses.append(dat.loc[i][AddrDict[InstCol]])
    
for i in dat.index:
    if i not in FirstAuthorIDs:
        OrderOfAuthors.append(i)
        for InstCol in InstColumns:
            if(dat.loc[i].notna()[InstCol] and not dat.loc[i][InstCol] in UniqueInstitutions):
                UniqueInstitutions.append(dat.loc[i][InstCol])
                Addresses.append(dat.loc[i][AddrDict[InstCol]])
           
UniqueInstitutions=np.array(UniqueInstitutions)
def InstitutionID(Name):
    return str(np.where(UniqueInstitutions==Name)[0][0])

UniqueNotes=dat.loc[OrderOfAuthors].Footnote.unique()

def NoteID(Name):
    return str(num2alpha[np.where(UniqueNotes==Name)[0][0]+1])


num2alpha = dict(zip(range(1, 27), string.ascii_lowercase))

In [6]:
InstColumns

array(['Institution1', 'Institution2', 'Institution3'], dtype=object)

In [7]:
# Function to make the author list for IOP journals (JCAP, JHEP, JINST...)

def MakeIOPAuthorList():
    authorlist=str()
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        instids=""
        for InstCol in InstColumns:
            if(entry.notna()[InstCol]):
                instids+=InstitutionID(entry[InstCol])+","
        if(len(str(entry.Footnote))>5):
            authorlist+=("\\author["+instids+NoteID(entry.Footnote)+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+",}\n")
            authorlist+=('\\note['+NoteID(entry.Footnote)+']{' + entry.Footnote.strip() + '}\n')
        else:
            authorlist+=("\\author["+instids[:-1]+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+",}\n")
        authorlist+=("%\n")
    authorlist+=("%\n")
    for i in range(0,len(UniqueInstitutions)):
        authorlist+="\\affiliation[" + str(InstitutionID(UniqueInstitutions[i])) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+Addresses[i]+"}\n"
        authorlist+=("%\n")
    return authorlist

In [8]:
# Function to make the author list for Elsevier journals (NIM...)

def MakeElsevierAuthorList():
    authorlist=str()
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        instids=""
        for InstCol in InstColumns:
            if(entry.notna()[InstCol]):
                instids+=InstitutionID(entry[InstCol])+","
        if(len(str(entry.Footnote))>5):
            authorlist+=("\\author["+instids[:-1]+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+"\corref{"+NoteID(entry.Footnote)+"}}\n")
            authorlist+=('\\cortext['+NoteID(entry.Footnote)+']{' + entry.Footnote.strip() + '}\n')
        else:
            authorlist+=("\\author["+instids[:-1]+"]{"+entry.Initial.strip()+"~"+entry.LastName.strip()+"}\n")
        authorlist+=("%\n")
    authorlist+=("%\n")
    for i in range(0,len(UniqueInstitutions)):
        authorlist+="\\address[" + InstitutionID(UniqueInstitutions[i]) + ']{\n' + UniqueInstitutions[i].replace('\\\\','\\\\\n')+", "+ Addresses[i]+"}\n"
        authorlist+=("%\n")
    return authorlist

In [9]:
# Function to make the author list for APS journals (PRD, PRL...)

def MakeAPSAuthorList():
    authorlist=str()
    for a in OrderOfAuthors:
        entry=dat.loc[a]
        authorlist+="\\author{"+entry.Initial.strip()+"~"+entry.LastName.strip()+",}\n"
        for InstCol in InstColumns:
            if(entry.notna()[InstCol]):
                authorlist+="\\affiliation{"+entry[InstCol].replace('\\\\','\\\\\n')+", "+entry[AddrDict[InstCol]]+"}\n"
        if(len(str(entry.Footnote))>5):
            authorlist+=('\\thanks{' + entry.Footnote.strip() + '}\n')
        authorlist+=("%\n")
    return authorlist

In [10]:
# Function to make the author list for arXiv

def MakeArXivAuthorList():
    authorlist=str("NEXT Collaboration: ")
    for a in OrderOfAuthors[:-1]:
        entry=dat.loc[a]
        authorlist+=entry.Initial.strip()+" "+entry.LastName.strip()+", "
    entry=dat.loc[OrderOfAuthors[-1]]
    authorlist=authorlist[:-2]+" and " + entry.Initial.strip()+" "+entry.LastName.strip()
    return authorlist

In [11]:
# Make author lists and store to .tex

f=open("authors_elsevier.tex",'w')
f.write(MakeElsevierAuthorList())
f.close()

f=open("authors_iop.tex",'w')
f.write(MakeIOPAuthorList())
f.close()

f=open("authors_aps.tex",'w')
f.write(MakeAPSAuthorList())
f.close()

f=open("authors_arxiv.tex",'w')
f.write(MakeArXivAuthorList())
f.close()

NameError: name 'GetOrderOfAuthors' is not defined